# Ensemble - RoBERTa-Title + RoBERTa-Body + Logistic Regression

We noticed that RoBERTa trained on title and on body tend to misclassify different articles and decided to try ensemble with these models plus logistic regression.

>**Note:** This was run in Google Colab, so there is no direct reference to the data. The data used was the same as in repository.

## Imports

In [ ]:
from google.colab import drive
import glob

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install simpletransformers -q

In [ ]:
import pandas as pd
import numpy as np
import torch 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import wandb

## Load data

In [ ]:
# CHANGE TO YOUR PATH
colab_resources_path = "/content/drive/My Drive/projectml"

In [ ]:
data_files = glob.glob(colab_resources_path + "/*.csv")
data_files += glob.glob(colab_resources_path + "/*.py")
for data_file in data_files:
  print('Copying file {} to colab root.'.format(data_file))
  !cp "$data_file" .

Copying file /content/drive/My Drive/projectml/random.csv to colab root.
Copying file /content/drive/My Drive/projectml/nam.csv to colab root.
Copying file /content/drive/My Drive/projectml/am.csv to colab root.
Copying file /content/drive/My Drive/projectml/am_additional.csv to colab root.
Copying file /content/drive/My Drive/projectml/missclassified_roberta_mcc.csv to colab root.
Copying file /content/drive/My Drive/projectml/miss_5models.csv to colab root.
Copying file /content/drive/My Drive/projectml/data_preprocess.py to colab root.


In [ ]:
from data_preprocess import getTrainData, getFullRowByIndex
train_data_title = getTrainData(include_random=True, random_as_2=False, shuffle=False, no_title=False, n_sentences=0) #for roberta on titles only
train_data_body = getTrainData(include_random=True, random_as_2=False, shuffle=False, no_title=True, n_sentences=-1) #for roberta on articles only
train_data_titleplus = getTrainData(include_random=True, random_as_2=False, shuffle=False, no_title=False, n_sentences=1) #for roberta on title+ 1 paraghaph only
train_data_all = getTrainData(include_random=True, random_as_2=False, shuffle=False, no_title=False, n_sentences=-1) #for roberta on all

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## 6-fold cross-validation for RoBERTa-title + RoBERTa-body + LR ensemble

`getProbabilitiesRoberta` takes classification model's output as an argument and returns numpy array with probability of each class

In [ ]:
from scipy.special import softmax
def getProbabilitiesRoberta(pred):
  
  return np.array([np.sum(softmax(j, axis=1), axis=0)/len(j) for j in pred])

Function for memory cleanup, useful if several models are trained in one notebook.

In [ ]:
import gc
def memory_cleanup():
    for obj in gc.get_objects():
        if torch.is_tensor(obj):
            del obj
    gc.collect()
    torch.cuda.empty_cache()
    ! rm -r outputs/

Specify classification model hyperparameters 

In [ ]:
model_args= ClassificationArgs(sliding_window=True)
model_args.num_train_epochs=4
model_args.save_best_model= True
model_args.tie_value = 1
model_args.batch_size = 16
model_args.learning_rate = 2e-5
model_args.overwrite_output_dir = True
model_args.max_seq_length = 512
model_args.max_grad_norm = 1
model_args.use_multiprocessing = True
model_args.manual_seed = 4
model_args.reprocess_input_data = True
model_args.evaluate_during_training = True
model_args.labels_list = [0, 1]

6-fold cross-validation, three models are trained at the same time.


*   RoBERTa-base is trained on article titles
*   RoBERTa-base is trained on article texts
*   Logistic Regression is trained on TfidVectorizer output


In [ ]:
n=6
seed=42
kf = KFold(n_splits=n, random_state=seed, shuffle=True)

mcc_lr, f1_lr = [], []
mcc_rb_body, f1_rb_body = [], []
mcc_rb_title, f1_rb_title = [], []
acc_eq, prec_eq, rec_eq, f1_eq, mcc_eq = [], [],[],[], []
acc_w, prec_w, rec_w, f1_w, mcc_w = [], [],[],[], []

miss_lr = np.array([])
miss_rb_body = np.array([])
miss_rb_title = np.array([])
miss_ens_eq = np.array([])
miss_ens_w = np.array([])

for train_index, val_index in kf.split(train_data_all): 

    #### LogReg
    train_df = train_data_all.iloc[train_index]
    val_df = train_data_all.iloc[val_index]

    cv = TfidfVectorizer(strip_accents='ascii', lowercase=True, stop_words='english')

    X_train_cv = cv.fit_transform(train_df.text)
    X_val_cv = cv.transform(val_df.text)
    
    lr = LogisticRegression(random_state=0, C=17, penalty='l2', max_iter=1000)
    lr.fit(X_train_cv, train_df.label)
    predictions_lr = lr.predict(X_val_cv)

    miss_lr = np.concatenate((miss_lr, val_df.iloc[np.where(val_df.label != predictions_lr)].index)).astype(int)

    f1_lr.append(f1_score(val_df.label, predictions_lr))
    mcc_lr.append(matthews_corrcoef(val_df.label, predictions_lr))

    #### RoBERTa body
    train_df = train_data_body.iloc[train_index]
    val_df = train_data_body.iloc[val_index]

    model = ClassificationModel('roberta', 'roberta-base', args=model_args)
    model.train_model(train_df, eval_df=val_df, acc=matthews_corrcoef)
    result, model_outputs_body, wrong_predictions = model.eval_model(val_df, acc=matthews_corrcoef) 

    predictions_rb_body = np.array([np.rint(np.mean(np.argmax(j, axis=1))) for j in model_outputs_body]).astype(int)

    miss_rb_body = np.concatenate((miss_rb_body, val_df.iloc[np.where(val_df.label != predictions_rb_body)].index)).astype(int)

    f1_rb_body.append(f1_score(val_df.label, predictions_rb_body))
    mcc_rb_body.append(matthews_corrcoef(val_df.label, predictions_rb_body))

    memory_cleanup()


    #### RoBERTa title
    train_df = train_data_title.iloc[train_index]
    val_df = train_data_title.iloc[val_index]

    model = ClassificationModel('roberta', 'roberta-base', args=model_args)
    model.train_model(train_df, eval_df=val_df, acc=matthews_corrcoef)
    result, model_outputs_title, wrong_predictions = model.eval_model(val_df, acc=matthews_corrcoef) 

    predictions_rb_title = np.array([np.rint(np.mean(np.argmax(j, axis=1))) for j in model_outputs_title]).astype(int)

    miss_rb_title = np.concatenate((miss_rb_title, val_df.iloc[np.where(val_df.label != predictions_rb_title)].index)).astype(int)

    f1_rb_title.append(f1_score(val_df.label, predictions_rb_title))
    mcc_rb_title.append(matthews_corrcoef(val_df.label, predictions_rb_title))

    memory_cleanup()

    ##### ENSEMBLE

    # weights for weighted ensemble are assigned based on cross-validation results
    w_lr = 0.93
    w_rb_body = 0.94
    w_rb_title = 0.93

    prob_lr = np.array(lr.predict_proba(X_val_cv))
    prob_rb_body = getProbabilitiesRoberta(model_outputs_body)
    prob_rb_title = getProbabilitiesRoberta(model_outputs_title)

    prob_lr = prob_lr[:, 0]
    prob_rb_body = prob_rb_body[:, 0]
    prob_rb_title = prob_rb_title[:, 0]

    prob_eq=(prob_lr+prob_rb_title+prob_rb_body)/3
    prob_w = (prob_lr*w_lr + prob_rb_body*w_rb_body+prob_rb_title*w_rb_title)/(w_lr+w_rb_title+w_rb_body)

    predictions_eq = np.where(prob_eq > 0.5, 0, 1)
    predictions_w = np.where(prob_w > 0.5, 0, 1)

    miss_ens_eq = np.concatenate((miss_ens_eq, val_df.iloc[np.where(val_df.label != predictions_eq)].index)).astype(int)
    miss_ens_w = np.concatenate((miss_ens_w, val_df.iloc[np.where(val_df.label != predictions_w)].index)).astype(int)

    # metrics for ensemble with equal weights
    acc_eq.append(accuracy_score(val_df.label, predictions_eq))
    prec_eq.append(precision_score(val_df.label, predictions_eq))
    rec_eq.append(recall_score(val_df.label, predictions_eq))
    f1_eq.append(f1_score(val_df.label, predictions_eq))
    mcc_eq.append(matthews_corrcoef(val_df.label, predictions_eq))

    # metrics for ensemble with different weights
    acc_w.append(accuracy_score(val_df.label, predictions_w))
    prec_w.append(precision_score(val_df.label, predictions_w))
    rec_w.append(recall_score(val_df.label, predictions_w))
    f1_w.append(f1_score(val_df.label, predictions_w))
    mcc_w.append(matthews_corrcoef(val_df.label, predictions_w))

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:966: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/usr/local/lib/python3.6/dist-packages/torch/distributed/distributed_c10d.py:126: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceCla

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:966: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/usr/local/lib/python3.6/dist-packages/torch/distributed/distributed_c10d.py:126: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceCla

...

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:966: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/usr/local/lib/python3.6/dist-packages/torch/distributed/distributed_c10d.py:126: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceCla

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:966: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/usr/local/lib/python3.6/dist-packages/torch/distributed/distributed_c10d.py:126: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "


Accuracy score equal:  0.959885562964014 Accuracy score weight:  0.959885562964014
Precision score equal:  0.9497345212843892 Precision score weight:  0.9497345212843892
Recall score equal:  0.9727059274216646 Recall score weight:  0.9727059274216646
F1 score eq:  0.9608639281203498 F1 score weight:  0.9608639281203498
F1 RB body score:  0.9515905784030895
F1 RB title score:  0.9313658581486742
F1 LR score:  0.9373184483141538
MCC ENS equal score:  0.9203156877975641
MCC ENS weight score:  0.9203156877975641
MCC RB body score:  0.9022887029673452
MCC RB title score:  0.86294510312981
MCC LR score:  0.8723273883124713


In [ ]:
print('Accuracy score equal: ', np.mean(acc_eq),',','Accuracy score weight: ', np.mean(acc_w))
print('Precision score equal: ', np.mean(prec_eq),',','Precision score weight: ', np.mean(prec_w))
print('Recall score equal: ', np.mean(rec_eq),',','Recall score weight: ', np.mean(rec_w))
print('F1 score eq: ', np.mean(f1_eq),',','F1 score weight: ', np.mean(f1_w))
print('F1 RB body score: ', np.mean(f1_rb_body))
print('F1 RB title score: ', np.mean(f1_rb_title))
print('F1 LR score: ', np.mean(f1_lr))
print('MCC ENS equal score: ', np.mean(mcc_eq),',','MCC ENS weight score: ', np.mean(mcc_w))
print('MCC RB body score: ', np.mean(mcc_rb_body))
print('MCC RB title score: ', np.mean(mcc_rb_title))
print('MCC LR score: ', np.mean(mcc_lr))

Accuracy score equal:  0.959885562964014 , Accuracy score weight:  0.959885562964014
Precision score equal:  0.9497345212843892 , Precision score weight:  0.9497345212843892
Recall score equal:  0.9727059274216646 , Recall score weight:  0.9727059274216646
F1 score eq:  0.9608639281203498 , F1 score weight:  0.9608639281203498
F1 RB body score:  0.9515905784030895
F1 RB title score:  0.9313658581486742
F1 LR score:  0.9373184483141538
MCC ENS equal score:  0.9203156877975641 , MCC ENS weight score:  0.9203156877975641
MCC RB body score:  0.9022887029673452
MCC RB title score:  0.86294510312981
MCC LR score:  0.8723273883124713


## Saving misclassified data

Create dataframe with articles misclassified at least by one model. Values in model columns are one if article is misclassified at least by one model, otherwise zero.

In [ ]:
miss_indexes = np.unique(np.concatenate((miss_rb_title, miss_rb_body, miss_lr, miss_ens_eq, miss_ens_w))).astype(int)

miss_data = getFullRowByIndex(miss_indexes)

miss_data['miss_lr'] = np.zeros(len(miss_data.index)).astype(int)
miss_data['miss_rb_title'] = np.zeros(len(miss_data.index)).astype(int)
miss_data['miss_rb_body'] = np.zeros(len(miss_data.index)).astype(int)
miss_data['miss_ens_eq'] = np.zeros(len(miss_data.index)).astype(int)
miss_data['miss_ens_w'] = np.zeros(len(miss_data.index)).astype(int)

miss_data.loc[miss_data.loc[miss_lr].index, 'miss_lr'] = 1
miss_data.loc[miss_data.loc[miss_rb_body].index, 'miss_rb_body'] = 1
miss_data.loc[miss_data.loc[miss_ens_w].index, 'miss_ens_w'] = 1
miss_data.loc[miss_data.loc[miss_rb_title].index, 'miss_rb_title'] = 1
miss_data.loc[miss_data.loc[miss_ens_eq].index, 'miss_ens_eq'] = 1

miss_data = miss_data.drop(columns=['assessor.1', 'article.1'])

try:
  miss_data.to_csv("/content/drive/My Drive/projectml/missclassified_lr_robertas_ens_mcc.csv")
except:
  miss_data.to_csv("/content/gdrive/MyDrive/projectml/missclassified_lr_robertas_ens_mcc.csv")

## Conlusion

Cross-validation results (**F1: 0.9609**) seem to be **better** than logistic regression or RoBERTa alone and are comparable with the results of RoBERTa trained on all article + logistic regression ensemble (**F1: 0.9583**).